### This tutorial will show how to interact with neo4j database using py2neo
1. It shows how to create a node and relationship using py2neo
2. How to query the data

In [1]:
from py2neo import Graph, Node, Relationship

In [2]:
#Graph instance, pass the credentials for you graph
graph=Graph(password="test123")

In [3]:
#To create Node using py2neo. Here creating 2 nodes and a relationship
alice = Node("Person", name="Alice")
bob = Node("Person", name="Bob")
alice_knows_bob = Relationship(alice, "KNOWS", bob)
graph.create(alice_knows_bob)

In [4]:
#To set the properties for the exising nodes
alice['age'] = 34
bob['age'] = 44

In [5]:
#To update the graph with the new nodes and relationships created above.
graph.push(alice)
graph.push(bob)

In [6]:
# To run the query. Below query fetches name and age properties of nodes with the label Person.
graph.run("MATCH (a:Person) \
          RETURN a.name, a.age").to_table()

a.name,a.age
Alice,34
Bob,44


In [7]:
#This will evaluate and retun the no of nodes that are matching the label Person
graph.evaluate("MATCH (a:Person) RETURN count(a)")

2

In [8]:
#to delete all nodes
graph.run("MATCH (a) DETACH DELETE a")

In [9]:
# To run the query. No nodes returned as we deleted them in the prev step.
list(graph.run("MATCH (n) RETURN n"))

[]

### Lets load movie graph that comes default with Neo4j using Neo4j browser and see how to query/update the data

In [10]:
## In the below code, we are using nodes.match to match the Person label and return the results in pythonic way 
# It will return the name and born properties of all the nodes which are matching. 
[(a["name"], a["born"]) for a in graph.nodes.match("Person")]

[('Al Pacino', 1940),
 ('Taylor Hackford', 1944),
 ('Tom Cruise', 1962),
 ('Jack Nicholson', 1937),
 ('Demi Moore', 1962),
 ('Kevin Bacon', 1958),
 ('Kiefer Sutherland', 1966),
 ('Noah Wyle', 1971),
 ('Cuba Gooding Jr.', 1968),
 ('Kevin Pollak', 1957),
 ('J.T. Walsh', 1943),
 ('James Marshall', 1967),
 ('Christopher Guest', 1948),
 ('Rob Reiner', 1947),
 ('Aaron Sorkin', 1961),
 ('Kelly McGillis', 1957),
 ('Val Kilmer', 1959),
 ('Anthony Edwards', 1962),
 ('Tom Skerritt', 1933),
 ('Meg Ryan', 1961),
 ('Tony Scott', 1944),
 ('Jim Cash', 1941),
 ('Renee Zellweger', 1969),
 ('Kelly Preston', 1962),
 ("Jerry O'Connell", 1974),
 ('Jay Mohr', 1970),
 ('Bonnie Hunt', 1961),
 ('Regina King', 1971),
 ('Jonathan Lipnicki', 1996),
 ('Cameron Crowe', 1957),
 ('River Phoenix', 1970),
 ('Corey Feldman', 1971),
 ('Wil Wheaton', 1972),
 ('John Cusack', 1966),
 ('Marshall Bell', 1942),
 ('Helen Hunt', 1963),
 ('Greg Kinnear', 1963),
 ('James L. Brooks', 1940),
 ('Annabella Sciorra', 1960),
 ('Max von S

In [11]:
## can save the resulting node from the match and use it next to fetch the relatioinships.
##It will first find the node that matches the name "Bonnie Hunt" and saves it variable bonnie
## It is then used to match the relationships for Bonnie Hunt for the relation - ACTED_IN
bonnie = graph.nodes.match("Person", name="Bonnie Hunt").first()
list(graph.relationships.match((bonnie, None), "ACTED_IN"))

[(Bonnie Hunt)-[:ACTED_IN {roles: ['Laurel Boyd']}]->(_42),
 (Bonnie Hunt)-[:ACTED_IN {roles: ['Jan Edgecomb']}]->(_135)]

In [12]:
#The below code shows how to create a Node and Relationship
jumanji = Node("Movie", "Jumanji")
ACTED_IN = Relationship.type("ACTED_IN")

graph.create(ACTED_IN(bonnie, jumanji))

graph.match((bonnie, jumanji)).first()

(Bonnie Hunt)-[:ACTED_IN {}]->(_178)

In [13]:
#The below query will fetch 5 movie titles and taglines and displays in dataframe 
graph.run("MATCH (m:Movie) RETURN m.title, m.tagline LIMIT 5").to_data_frame()

,m.tagline,m.title
0,"In the heart of the nation's capital, in a cou...",A Few Good Men
1,"I feel the need, the need for speed.",Top Gun
2,The rest of his life begins now.,Jerry Maguire
3,"For some, it's the last real taste of innocenc...",Stand By Me
4,A comedy from the heart that goes for the throat.,As Good as It Gets


#### Another way is to use Object-Graph Mapping API of py2neo.

In [14]:
## below code shows how to leverage this API to query the data
from py2neo.ogm import GraphObject, Property

class Person(GraphObject):
    name = Property()
    born = Property()
    
[(a.name, a.born) for a in Person.match(graph)]

[('Al Pacino', 1940),
 ('Taylor Hackford', 1944),
 ('Tom Cruise', 1962),
 ('Jack Nicholson', 1937),
 ('Demi Moore', 1962),
 ('Kevin Bacon', 1958),
 ('Kiefer Sutherland', 1966),
 ('Noah Wyle', 1971),
 ('Cuba Gooding Jr.', 1968),
 ('Kevin Pollak', 1957),
 ('J.T. Walsh', 1943),
 ('James Marshall', 1967),
 ('Christopher Guest', 1948),
 ('Rob Reiner', 1947),
 ('Aaron Sorkin', 1961),
 ('Kelly McGillis', 1957),
 ('Val Kilmer', 1959),
 ('Anthony Edwards', 1962),
 ('Tom Skerritt', 1933),
 ('Meg Ryan', 1961),
 ('Tony Scott', 1944),
 ('Jim Cash', 1941),
 ('Renee Zellweger', 1969),
 ('Kelly Preston', 1962),
 ("Jerry O'Connell", 1974),
 ('Jay Mohr', 1970),
 ('Bonnie Hunt', 1961),
 ('Regina King', 1971),
 ('Jonathan Lipnicki', 1996),
 ('Cameron Crowe', 1957),
 ('River Phoenix', 1970),
 ('Corey Feldman', 1971),
 ('Wil Wheaton', 1972),
 ('John Cusack', 1966),
 ('Marshall Bell', 1942),
 ('Helen Hunt', 1963),
 ('Greg Kinnear', 1963),
 ('James L. Brooks', 1940),
 ('Annabella Sciorra', 1960),
 ('Max von S

In [15]:
#To return the first matching result
graph.nodes.match("Person", name="Keanu Reeves").first()

(_211:Person {born: 1964, name: 'Keanu Reeves'})

### Leveraging py2neo. matching module. It provides functionality to match nodes and relationships according to certain criteria

In [16]:
from py2neo import NodeMatcher

matcher = NodeMatcher(graph)

In [17]:
#Querying using Matching module
matcher.match("Person", name="Keanu Reeves").first()

(_211:Person {born: 1964, name: 'Keanu Reeves'})

In [18]:
#to check for partial match. The below query will find all the nodes with names start with 'B'
list(matcher.match("Person").where("_.name =~ 'B.*'" ))

[(_47:Person {born: 1961, name: 'Bonnie Hunt'}),
 (_80:Person {born: 1953, name: 'Bill Pullman'}),
 (_87:Person {born: 1948, name: 'Billy Crystal'}),
 (_89:Person {born: 1949, name: 'Bruno Kirby'}),
 (_93:Person {born: 1970, name: 'Brooke Langton'}),
 (_125:Person {born: 1967, name: 'Ben Miles'}),
 (_151:Person {born: 1955, name: 'Bill Paxton'})]

In [19]:
#can use order by and limit 
list(matcher.match("Person").where("_.name =~ 'B.*'" ).order_by("_.name").limit(3))

[(_125:Person {born: 1967, name: 'Ben Miles'}),
 (_151:Person {born: 1955, name: 'Bill Paxton'}),
 (_80:Person {born: 1953, name: 'Bill Pullman'})]

In [20]:
#to count the no of matches
len(matcher.match("Person").where("_.name =~ 'B.*'" ))

7